In [11]:
import pandas as pd
import nltk
import pickle
nltk.download('stopwords')
nltk.download('wordnet')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
df=pd.read_csv('spam', sep='\t', names=['Label', 'Messages'])
df.head()


,Label,Messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
df.shape

(5572, 2)

In [14]:
df.isnull().sum()

Label       0
Messages    0
dtype: int64

In [15]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnl=WordNetLemmatizer()
corpus=[]
for i in range (0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['Messages'][i])
    review = review.lower()
    review = review.split()
    
    review = [wnl.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
corpus[1]

'ok lar joking wif u oni'

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer(max_features=2500)
x=cv.fit_transform(corpus).toarray()
y=pd.get_dummies(df['Label'])
y=y.iloc[:,1].values

In [18]:
pickle.dump(cv, open('tfidf.pkl', 'wb'))


In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [20]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
pickle.dump(clf, open('model.pkl', 'wb'))